PRIMER PASO: BARAJEAR DATOS

In [6]:
import pandas as pd
import random

# Leer el dataset original desde la carpeta data
df = pd.read_csv('data/initial_data.csv')

print(f"Dataset original cargado: {len(df)} noticias")
print(f"Distribución de sentimientos:")
print(df['Sentiment'].value_counts())
print("\n" + "="*60)

# Crear una copia para las noticias correctas (50%)
df_correctas = df.copy()
df_correctas['Etiqueta'] = 'correcta'

# Separar por sentimientos para mezclas
df_positive = df[df['Sentiment'] == 'positive']
df_negative = df[df['Sentiment'] == 'negative']

print(f"\nNoticias positivas disponibles: {len(df_positive)}")
print(f"Noticias negativas disponibles: {len(df_negative)}")

# Función para extraer fragmentos de una noticia
def extraer_fragmentos(noticia):
    """
    Extrae fragmentos de una noticia dividiéndola por comas o puntos
    """
    # Primero intentar dividir por comas
    fragmentos = [p.strip() for p in noticia.split(',') if p.strip()]
    
    # Si no hay suficientes fragmentos, dividir por puntos
    if len(fragmentos) <= 1:
        fragmentos = [p.strip() for p in noticia.split('.') if p.strip()]
    
    return fragmentos

# Función para crear mezclas de sentimientos OPUESTOS
def crear_mezclas_opuestas(df_positive, df_negative, num_mezclas):
    """
    Crea mezclas incorrectas combinando noticias positivas con negativas
    """
    noticias_mezcladas = []
    
    print(f"\nCreando {num_mezclas} mezclas de sentimientos opuestos...")
    
    for i in range(num_mezclas):
        if (i + 1) % 1000 == 0:
            print(f"  Procesadas {i + 1}/{num_mezclas}...")
        
        # Seleccionar una noticia positiva y una negativa
        idx_pos = random.randint(0, len(df_positive) - 1)
        idx_neg = random.randint(0, len(df_negative) - 1)
        
        noticia_pos = df_positive.iloc[idx_pos]['Sentence']
        noticia_neg = df_negative.iloc[idx_neg]['Sentence']
        
        # Extraer fragmentos
        frag_pos = extraer_fragmentos(noticia_pos)
        frag_neg = extraer_fragmentos(noticia_neg)
        
        # Seleccionar 1-2 fragmentos de cada una
        if len(frag_pos) > 0 and len(frag_neg) > 0:
            num_pos = min(random.randint(1, 2), len(frag_pos))
            num_neg = min(random.randint(1, 2), len(frag_neg))
            
            fragmentos_seleccionados = (
                random.sample(frag_pos, num_pos) + 
                random.sample(frag_neg, num_neg)
            )
            
            # Mezclar el orden
            random.shuffle(fragmentos_seleccionados)
            
            noticia_mezclada = ', '.join(fragmentos_seleccionados)
            if noticia_mezclada and noticia_mezclada[-1] not in '.!?':
                noticia_mezclada += '.'
            
            # Asignar sentimiento aleatorio entre los dos opuestos
            sentiment = random.choice(['positive', 'negative'])
            
            noticias_mezcladas.append({
                'Sentence': noticia_mezclada,
                'Sentiment': sentiment,
                'Etiqueta': 'incorrecta'
            })
    
    return pd.DataFrame(noticias_mezcladas)

# Crear el mismo número de noticias incorrectas que correctas
num_mezclas = len(df_correctas)
df_incorrectas = crear_mezclas_opuestas(df_positive, df_negative, num_mezclas)

# Combinar ambos datasets
df_final = pd.concat([df_correctas, df_incorrectas], ignore_index=True)

# Mezclar aleatoriamente el dataset final
df_final = df_final.sample(frac=1, random_state=42).reset_index(drop=True)

# Mostrar estadísticas
print(f"\n{'='*60}")
print(f"ESTADÍSTICAS DEL DATASET FINAL")
print(f"{'='*60}")
print(f"Total de filas: {len(df_final)}")
print(f"Noticias correctas: {len(df_final[df_final['Etiqueta'] == 'correcta'])} ({len(df_final[df_final['Etiqueta'] == 'correcta'])/len(df_final)*100:.1f}%)")
print(f"Noticias incorrectas: {len(df_final[df_final['Etiqueta'] == 'incorrecta'])} ({len(df_final[df_final['Etiqueta'] == 'incorrecta'])/len(df_final)*100:.1f}%)")

print(f"\nDistribución de sentimientos en noticias CORRECTAS:")
print(df_final[df_final['Etiqueta'] == 'correcta']['Sentiment'].value_counts())

print(f"\nDistribución de sentimientos en noticias INCORRECTAS:")
print(df_final[df_final['Etiqueta'] == 'incorrecta']['Sentiment'].value_counts())

# Mostrar ejemplos
print(f"\n{'='*60}")
print(f"EJEMPLOS")
print(f"{'='*60}")

print("\n[NOTICIA CORRECTA]")
ejemplo_correcta = df_final[df_final['Etiqueta'] == 'correcta'].iloc[0]['Sentence']
print(f"Sentimiento: {df_final[df_final['Etiqueta'] == 'correcta'].iloc[0]['Sentiment']}")
print(ejemplo_correcta[:300] + ("..." if len(ejemplo_correcta) > 300 else ""))

print("\n[NOTICIA INCORRECTA - Ejemplo 1]")
ejemplo_inc1 = df_final[df_final['Etiqueta'] == 'incorrecta'].iloc[0]['Sentence']
print(f"Sentimiento: {df_final[df_final['Etiqueta'] == 'incorrecta'].iloc[0]['Sentiment']}")
print(ejemplo_inc1[:300] + ("..." if len(ejemplo_inc1) > 300 else ""))

print("\n[NOTICIA INCORRECTA - Ejemplo 2]")
ejemplo_inc2 = df_final[df_final['Etiqueta'] == 'incorrecta'].iloc[50]['Sentence']
print(f"Sentimiento: {df_final[df_final['Etiqueta'] == 'incorrecta'].iloc[50]['Sentiment']}")
print(ejemplo_inc2[:300] + ("..." if len(ejemplo_inc2) > 300 else ""))

print("\n[NOTICIA INCORRECTA - Ejemplo 3]")
ejemplo_inc3 = df_final[df_final['Etiqueta'] == 'incorrecta'].iloc[100]['Sentence']
print(f"Sentimiento: {df_final[df_final['Etiqueta'] == 'incorrecta'].iloc[100]['Sentiment']}")
print(ejemplo_inc3[:300] + ("..." if len(ejemplo_inc3) > 300 else ""))

# Guardar el dataset final
output_path = 'data/dataset_mezclado_final.csv'
df_final.to_csv(output_path, index=False, encoding='utf-8')
print(f"\n{'='*60}")
print(f"✓ Dataset guardado como '{output_path}'")
print(f"{'='*60}")

Dataset original cargado: 5842 noticias
Distribución de sentimientos:
Sentiment
neutral     3130
positive    1852
negative     860
Name: count, dtype: int64


Noticias positivas disponibles: 1852
Noticias negativas disponibles: 860

Creando 5842 mezclas de sentimientos opuestos...
  Procesadas 1000/5842...
  Procesadas 2000/5842...
  Procesadas 3000/5842...
  Procesadas 4000/5842...
  Procesadas 5000/5842...

ESTADÍSTICAS DEL DATASET FINAL
Total de filas: 11684
Noticias correctas: 5842 (50.0%)
Noticias incorrectas: 5842 (50.0%)

Distribución de sentimientos en noticias CORRECTAS:
Sentiment
neutral     3130
positive    1852
negative     860
Name: count, dtype: int64

Distribución de sentimientos en noticias INCORRECTAS:
Sentiment
positive    2939
negative    2903
Name: count, dtype: int64

EJEMPLOS

[NOTICIA CORRECTA]
Sentimiento: neutral
The presentation material can be viewed on the company 's website in English after the conference .

[NOTICIA INCORRECTA - Ejemplo 1]
Sentimiento: neg

TRADUCCIÓN

In [2]:
# Instalar librerías necesarias
!pip install deep-translator pandas tqdm

import pandas as pd
from deep_translator import GoogleTranslator
import random
from tqdm import tqdm
import time
import os

# Leer el dataset original
df = pd.read_csv('data/dataset_mezclado_final.csv')

# Definir los idiomas a los que traducir (códigos ISO)
idiomas = ['en', 'fr', 'de', 'it', 'pt', 'ca', 'eu', 'gl']  # inglés, francés, alemán, italiano, portugués, catalán, euskera, gallego

# Diccionario para nombres completos de idiomas
nombres_idiomas = {
    'es': 'español',
    'en': 'inglés',
    'fr': 'francés',
    'de': 'alemán',
    'it': 'italiano',
    'pt': 'portugués',
    'ca': 'catalán',
    'eu': 'euskera',
    'gl': 'gallego'
}

# Crear una copia del dataframe
df_traducido = df.copy()

# Crear nueva columna para el idioma
df_traducido['Idioma'] = ''

# Función para traducir con manejo de errores
def traducir_texto(texto, idioma_destino):
    try:
        if idioma_destino == 'es':  # Si es español, no traducir
            return texto
        translator = GoogleTranslator(source='es', target=idioma_destino)
        traduccion = translator.translate(texto)
        time.sleep(0.5)  # Pausa para evitar límites de la API
        return traduccion
    except Exception as e:
        print(f"Error traduciendo a {idioma_destino}: {e}")
        return texto  # Si falla, devolver el texto original

# Traducir SOLO la primera columna (Sentence) de cada fila a un idioma aleatorio
print("Iniciando traducción del dataset...")
print(f"Total de filas a procesar: {len(df_traducido)}")

for idx in tqdm(range(len(df_traducido))):
    # Seleccionar un idioma aleatorio (incluyendo español para mantener algunas filas originales)
    idioma_elegido = random.choice(idiomas + ['es', 'es', 'es'])  # Mayor probabilidad de español
    
    # Traducir SOLO la columna 'Sentence' (primera columna)
    # Las columnas 'Sentiment' y 'Etiqueta' permanecen iguales
    texto_original = df_traducido.loc[idx, 'Sentence']
    df_traducido.loc[idx, 'Sentence'] = traducir_texto(texto_original, idioma_elegido)
    
    # Añadir el idioma a la nueva columna
    df_traducido.loc[idx, 'Idioma'] = nombres_idiomas[idioma_elegido]



# Guardar el nuevo dataset
output_path = 'data/dataset_multiidioma.csv'
df_traducido.to_csv(output_path, index=False, encoding='utf-8')

print(f"\n✓ Dataset traducido guardado en: {output_path}")
print(f"Total de filas: {len(df_traducido)}")
print(f"\nEstructura del dataset:")
print(df_traducido.head())
print(f"\nDistribución de idiomas:")
print(df_traducido['Idioma'].value_counts())

Defaulting to user installation because normal site-packages is not writeable
Iniciando traducción del dataset...
Total de filas a procesar: 11684


  0%|          | 8/11684 [00:11<4:37:29,  1.43s/it]


KeyboardInterrupt: 

1. Análisis Exploratorio de Datos (EDA)

In [4]:
!pip install seaborn


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Python313\python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
import numpy as np

# 1. Carga y manejo del error de ruta
print("Iniciando Análisis Exploratorio de Datos...")
try:
    # Corrección: Intenta leer el archivo directamente desde el directorio actual
    df = pd.read_csv('data/dataset_multiidioma.csv', encoding='utf-8')
except FileNotFoundError:
    print("\nERROR: No se encontró 'dataset_multiidioma.csv'.")
    exit()

print("=" * 60)
print("✨ RESUMEN DEL DATASET ✨")
print("=" * 60)

# 2. Estructura, Tipos de Datos y Nulos (Combinado en una sola llamada)
print("\n1. ESTRUCTURA Y CONTEO DE VALORES NO NULOS (df.info()):")
df.info()

print("\n2. ESTADÍSTICAS DESCRIPTIVAS (df.describe(include='all')):")
print("-" * 60)
# Muestra estadísticas resumidas para todas las columnas
print(df.describe(include='all'))

# 3. Distribución de Variables Categóricas Clave
print("\n3. DISTRIBUCIONES CLAVE (Conteo y Porcentaje):")
for col in ['Sentiment', 'Idioma']:
    # Combina Conteo Absoluto y Porcentaje en una sola tabla limpia
    counts = df[col].value_counts().rename('Conteo')
    percents = df[col].value_counts(normalize=True).mul(100).round(2).rename('Porcentaje (%)')
    print(f"\n--- Distribución de: {col} ---")
    print(pd.concat([counts, percents], axis=1))

# 4. Análisis de la Longitud de las Oraciones
# Cálculo eficiente: cuenta palabras usando métodos vectorizados de pandas (.str)
df['sentence_length'] = df['Sentence'].str.split().str.len()
print("\n4. ESTADÍSTICAS DE LONGITUD DE ORACIONES (En palabras):")
print("-" * 60)
# Utiliza .describe() para obtener media, mediana, min, max, std en una línea
print(df['sentence_length'].describe().round(2))

Iniciando Análisis Exploratorio de Datos...
✨ RESUMEN DEL DATASET ✨

1. ESTRUCTURA Y CONTEO DE VALORES NO NULOS (df.info()):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11684 entries, 0 to 11683
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Sentence   11684 non-null  object
 1   Sentiment  11684 non-null  object
 2   Etiqueta   11684 non-null  object
 3   Idioma     11684 non-null  object
dtypes: object(4)
memory usage: 365.2+ KB

2. ESTADÍSTICAS DESCRIPTIVAS (df.describe(include='all')):
------------------------------------------------------------
         Sentence Sentiment  Etiqueta   Idioma
count       11684     11684     11684    11684
unique      11602         3         2        9
top     700 Model  positive  correcta  español
freq            3      4791      5842     3176

3. DISTRIBUCIONES CLAVE (Conteo y Porcentaje):

--- Distribución de: Sentiment ---
           Conteo  Porcentaje (%)
Sentiment        

2. Visualizaciones del Dataset

In [4]:
# ====================================================================
# CÓDIGO DE ANÁLISIS EXPLORATORIO ESENCIAL (Sin comandos de instalación)
# ====================================================================

# Importaciones
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# --- SIMULACIÓN DE DATOS (REEMPLAZA ESTA SECCIÓN CON TU CARGA DE DATOS) ---
# Necesario para que el código corra sin errores si no tienes un archivo
np.random.seed(42)
data_size = 300
data = {
    'Sentiment': np.random.choice(['positive', 'negative', 'neutral'], size=data_size, p=[0.45, 0.35, 0.20]),
    'Idioma': np.random.choice(['en', 'es', 'fr'], size=data_size, p=[0.6, 0.3, 0.1]),
    'sentence_length': (np.random.normal(loc=15, scale=5, size=data_size)).astype(int).clip(min=5)
}
df = pd.DataFrame(data)
df.loc[df['Sentiment'] == 'negative', 'sentence_length'] += 3
df.loc[df['Sentiment'] == 'positive', 'sentence_length'] -= 1
df['sentence_length'] = df['sentence_length'].clip(min=5) 
# -------------------------------------------------------------------------


# Configuración de estilo
sns.set_theme(style="whitegrid", palette="husl")

print("\n" + "=" * 60)
print("GENERANDO LAS 3 VISUALIZACIONES MÁS LÓGICAS PARA SENTIMIENTOS")
print("=" * 60)

# Crear figura con 3 subplots en una sola fila
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Visualizaciones Clave: Sentimiento y Longitud de Oración', fontsize=16, fontweight='bold')

# 1. Distribución de Sentimientos (Gráfico de Conteo)
sns.countplot(x='Sentiment', data=df, ax=axes[0],
              order=df['Sentiment'].value_counts().index)
axes[0].set_title('1. Distribución de Sentimientos (Objetivo)', fontweight='bold')
axes[0].set_xlabel('Sentimiento')
axes[0].set_ylabel('Frecuencia')

# 2. Distribución de Longitud de Oraciones (Histograma)
sns.histplot(df['sentence_length'], bins=30, kde=True, ax=axes[1])
media = df['sentence_length'].mean()
axes[1].axvline(media, color='red', linestyle='--', label=f'Media: {media:.1f}')
axes[1].set_title('2. Distribución de Longitud (Numérica)', fontweight='bold')
axes[1].set_xlabel('Número de Palabras')
axes[1].legend()

# 3. Longitud de Oraciones por Sentimiento (Boxplot)
sns.boxplot(x='Sentiment', y='sentence_length', data=df, ax=axes[2])
axes[2].set_title('3. Relación: Longitud vs. Sentimiento', fontweight='bold')
axes[2].set_xlabel('Sentimiento')
axes[2].set_ylabel('Número de Palabras')

plt.tight_layout(rect=[0, 0.03, 1, 0.9])

plt.savefig('analisis_esencial_sentimientos.png', dpi=300)
print("\n✓ Gráficos esenciales guardados como 'analisis_esencial_sentimientos.png'")
plt.show()

# --- Estadísticas de la relación clave ---
print("\n" + "=" * 60)
print("ESTADÍSTICAS DE LONGITUD POR SENTIMIENTO")
print("=" * 60)
print(df.groupby('Sentiment')['sentence_length'].describe())


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\aleja\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\aleja\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1043, in launch_instance
    app.start()
  File "C:\Users\aleja\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 728, in start
    self.io_lo

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

3. Preprocesamiento de Datos - Tokenización y Limpieza

In [8]:
import re
import pandas as pd
import string
import nltk

# --- GESTIÓN DE NLTK ESENCIAL ---
# El error está aquí, pero lo vamos a ignorar con el fallback en el código.
try:
    print("Verificando recursos NLTK esenciales...")
    # Intentamos descargar, pero el error de LookupError nos dice que a veces falla.
    nltk.download('punkt', quiet=True, raise_on_error=False)
    nltk.download('stopwords', quiet=True, raise_on_error=False)
    print("✓ Recursos NLTK (punkt, stopwords) listos.")
except Exception:
    print("ATENCIÓN: La descarga de recursos de NLTK falló. Usaremos tokenización simple.")

from nltk.corpus import stopwords
# La importación de word_tokenize es necesaria, pero la usaremos dentro de un try/except.
from nltk.tokenize import word_tokenize 
# -----------------------------------------------------------------------

# Función de tokenización robusta para usar como fallback
def simple_tokenize(text):
    """Tokeniza de forma básica usando regex si NLTK falla."""
    return re.findall(r"[\w']+|[.,!?;]", text.lower())

print("\n" + "=" * 80)
print("PREPROCESAMIENTO DE DATOS SIMPLE Y ROBUSTO")
print("=" * 80)

# --- SIMULACIÓN DE DATOS (REEMPLAZA ESTO CON TU CÓDIGO DE CARGA) ---
try:
    if 'df' not in locals():
        print("Creando DataFrame de simulación (df)...")
        data = {
            'Sentence': [
                "This is a great movie! Check out http://link.com #positive",
                "¡Qué mal día! No me gustó nada. @user",
                "neutral sentence with a question mark?",
                "This is a very very long sentence with many unnecessary words and stop words.",
                "La vida es bella. Es cierto.",
            ]
        }
        df = pd.DataFrame(data)
except NameError:
    pass
# -------------------------------------------------------------------------


# Cargar stopwords de múltiples idiomas
try:
    STOPWORDS_ES = set(stopwords.words('spanish'))
    STOPWORDS_EN = set(stopwords.words('english'))
    STOPWORDS_ALL = STOPWORDS_ES.union(STOPWORDS_EN) 
except LookupError:
    STOPWORDS_ALL = set()


# Crear una copia del DataFrame para el proceso
df_processed = df.copy()


## 🎯 FUNCIÓN PRINCIPAL DE PREPROCESAMIENTO
def preprocess_text(text, stopwords_set, min_len=2):
    """
    Realiza limpieza, tokenización, minúsculas y eliminación de stopwords/puntuación
    en una sola función.
    """
    if pd.isna(text) or not str(text).strip():
        return []
    
    text = str(text)
    
    # 1. LIMPIEZA BÁSICA
    text = re.sub(r'http\S+|www\S+|https\S+|@\w+|#\w+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\s+', ' ', text).strip()
    
    # 2. TOKENIZACIÓN y CASE FOLDING
    try:
        # Intenta usar NLTK (más preciso)
        tokens = word_tokenize(text.lower()) 
    except LookupError:
        # Usa el tokenizer simple (robusto)
        tokens = simple_tokenize(text)

    # 3. ELIMINACIÓN DE PUNTUACIÓN, STOPWORDS y tokens muy cortos
    tokens_final = [
        token for token in tokens
        if token not in string.punctuation 
        and token not in stopwords_set
        and len(token) > min_len
    ]
    
    return tokens_final


# ====================================================================
# 1. APLICAR PREPROCESAMIENTO Y CREAR COLUMNA DE COMPARACIÓN
# ====================================================================

print("\n1. APLICANDO PROCESO DE LIMPIEZA...")
print("-" * 80)

df_processed['tokens_processed'] = df_processed['Sentence'].apply(
    lambda x: preprocess_text(x, STOPWORDS_ALL)
)

# ⭐️ SOLUCIÓN AL LOOKUPERROR: Usar el tokenizer robusto (simple_tokenize) para
# crear la columna de tokens originales, evitando la dependencia directa de NLTK.
df_processed['tokens_original'] = df_processed['Sentence'].apply(
     lambda x: simple_tokenize(str(x)) if not pd.isna(x) else []
)

print("✓ Preprocesamiento completado en la columna 'tokens_processed'")


# ====================================================================
# 2. VISUALIZACIÓN Y RESULTADO FINAL
# ====================================================================

print("\n" + "=" * 80)
print("RESULTADOS DEL PREPROCESAMIENTO Y ESTADÍSTICAS")
print("=" * 80)

## Muestra la columna de resultado
print("\n✅ Columna de tokens procesados (Primeras 5 filas):")
print(df_processed[['Sentence', 'tokens_processed']].head())

print("-" * 80)

## Muestra un ejemplo de comparación
if len(df_processed) > 0:
    print("\nEjemplo de transformación de la primera fila:")
    print(f"  Texto Original: {df['Sentence'].iloc[0]}")
    print(f"  Tokens Finales: {df_processed['tokens_processed'].iloc[0]}")

print("-" * 80)

## Estadísticas de reducción
df_processed['num_tokens_original'] = df_processed['tokens_original'].apply(len)
df_processed['num_tokens_processed'] = df_processed['tokens_processed'].apply(len)

mean_original = df_processed['num_tokens_original'].mean()
mean_processed = df_processed['num_tokens_processed'].mean()
reduction_percentage = ((mean_original - mean_processed) / mean_original * 100) if mean_original > 0 else 0

print(f"\nEstadísticas de Longitud:")
print(f"  Promedio de tokens originales: {mean_original:.2f}")
print(f"  Promedio de tokens procesados: {mean_processed:.2f}")
print(f"  Reducción de ruido promedio: **{reduction_percentage:.2f}%**")

# Guardar resultado final
df_processed.to_csv('datos_preprocesados_simple.csv', index=False)
print(f"\n✓ Datos guardados en 'datos_preprocesados_simple.csv'")

Verificando recursos NLTK esenciales...
✓ Recursos NLTK (punkt, stopwords) listos.

PREPROCESAMIENTO DE DATOS SIMPLE Y ROBUSTO

1. APLICANDO PROCESO DE LIMPIEZA...
--------------------------------------------------------------------------------
✓ Preprocesamiento completado en la columna 'tokens_processed'

RESULTADOS DEL PREPROCESAMIENTO Y ESTADÍSTICAS

✅ Columna de tokens procesados (Primeras 5 filas):
                                            Sentence  \
0  The presentation material can be viewed on the...   
1  Scansione di fine giornata: ipercomprato stoca...   
2  EGUNERAZIOA 1-Lloydsek 945 lanpostu moztuko di...   
3  Incap Contract Manufacturing Services Private ...   
4  mas aproximando-se da linha de resistência de ...   

                                    tokens_processed  
0  [presentation, material, viewed, company, webs...  
1  [scansione, fine, giornata, ipercomprato, stoc...  
2  [egunerazioa, lloydsek, 945, lanpostu, moztuko...  
3  [incap, contract, manufacturing,

4. Lemmatization y Stemming

In [10]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
import pandas as pd
import nltk
import numpy as np # Necesario para la simulación

# --- GESTIÓN NLTK ESENCIAL ---
try:
    print("Verificando recursos NLTK esenciales (wordnet)...")
    nltk.download('wordnet', quiet=True, raise_on_error=False)
    print("✓ Recurso 'wordnet' listo.")
except Exception:
    print("⚠️ ATENCIÓN: El recurso 'wordnet' de NLTK no está disponible.")

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
# -----------------------------------------------------------

print("\n" + "=" * 60)
print("LEMMATIZATION Y STEMMING CORREGIDO")
print("=" * 60)

# --- SIMULACIÓN DEL DATAFRAME NECESARIO ---
# ⚠️ ESTA SIMULACIÓN CREA EL DATAFRAME CON LA COLUMNA REQUERIDA. 
# Si estás ejecutando esto DESPUÉS del código de preprocesamiento anterior,
# puedes ELIMINAR O COMENTAR esta sección.
try:
    if 'df_processed' not in locals() or 'tokens_processed' not in df_processed.columns:
        print("Creando DataFrame de simulación con columna 'tokens_processed'...")
        df_processed = pd.DataFrame({
            'tokens_processed': [
                ['presentation', 'material', 'viewed', 'company', 'websites', 'running'],
                ['scansione', 'fine', 'giornata', 'ipercomprato', 'testing'],
                ['lloydsek', 'lanpostu', 'moztuko', 'ditu', 'runs'],
                ['incap', 'contract', 'manufacturing', 'services', 'fairly'],
                ['aproximando', 'linha', 'resistencia', 'culpa', 'acoes'],
            ]
        })
except NameError:
    pass
# -----------------------------------------------------------


# Inicializar herramientas
porter_stemmer = PorterStemmer()
word_lemmatizer = WordNetLemmatizer()

# Crear lista de tokens de ejemplo para demostración
sample_tokens = ['running', 'runs', 'ran', 'easily', 'fairly', 'testing', 'corpora']


## 1. STEMMING (Porter Stemmer)
print("\n1. STEMMING (Porter Stemmer - Inglés)")
print("-" * 60)

# ⭐️ CORRECCIÓN: Usamos 'tokens_processed' en lugar de 'tokens_no_stop'
df_processed['tokens_stemmed'] = df_processed['tokens_processed'].apply(
    lambda tokens: [porter_stemmer.stem(token) for token in tokens]
)
print("✓ Stemming completado. Columna: 'tokens_stemmed'")


## 2. LEMMATIZATION (WordNet Lemmatizer)
print("\n2. LEMMATIZATION (WordNet Lemmatizer)")
print("-" * 60)

# ⭐️ CORRECCIÓN: Usamos 'tokens_processed' en lugar de 'tokens_no_stop'
df_processed['tokens_lemmatized'] = df_processed['tokens_processed'].apply(
    lambda tokens: [word_lemmatizer.lemmatize(token, pos='v') for token in tokens]
)
print("✓ Lemmatization completada. Columna: 'tokens_lemmatized'")


## 3. COMPARACIÓN y VOCABULARIO (Usamos 'tokens_processed' como base)
print("\n3. COMPARACIÓN DE RESULTADOS Y VOCABULARIO")
print("-" * 60)

# Generamos la lista base para la comparación
base_tokens = df_processed['tokens_processed']

# Mostrar comparación del primer texto
if not df_processed.empty:
    original = base_tokens.iloc[0][:3]
    stemmed = df_processed['tokens_stemmed'].iloc[0][:3]
    lemmatized = df_processed['tokens_lemmatized'].iloc[0][:3]
    
    print(f"{'Token Original':<20} {'Stemming':<20} {'Lemmatization':<20}")
    print("-" * 60)
    for i in range(min(len(original), 3)):
        print(f"{original[i]:<20} {stemmed[i]:<20} {lemmatized[i]:<20}")

# Calcular y comparar el tamaño del vocabulario
vocab_original = set(token for tokens in base_tokens for token in tokens)
vocab_stemmed = set(token for tokens in df_processed['tokens_stemmed'] for token in tokens)
vocab_lemmatized = set(token for tokens in df_processed['tokens_lemmatized'] for token in tokens)

print(f"\nTamaño del vocabulario base: {len(vocab_original)}")
print(f"Tamaño después de Stemming: {len(vocab_stemmed)} ({((len(vocab_original) - len(vocab_stemmed)) / len(vocab_original) * 100):.2f}% reducción)")
print(f"Tamaño después de Lemmatization: {len(vocab_lemmatized)} ({((len(vocab_original) - len(vocab_lemmatized)) / len(vocab_original) * 100):.2f}% reducción)")


# 4. GUARDAR DATOS FINALES
df_processed['text_stemmed'] = df_processed['tokens_stemmed'].apply(' '.join)
df_processed['text_lemmatized'] = df_processed['tokens_lemmatized'].apply(' '.join)
df_processed['text_processed_base'] = base_tokens.apply(' '.join)

df_processed.to_csv('datos_preprocesados_completo_corregido.csv', index=False)
print(f"\n✓ Datos guardados en 'datos_preprocesados_completo_corregido.csv'")

Verificando recursos NLTK esenciales (wordnet)...
✓ Recurso 'wordnet' listo.

LEMMATIZATION Y STEMMING CORREGIDO

1. STEMMING (Porter Stemmer - Inglés)
------------------------------------------------------------
✓ Stemming completado. Columna: 'tokens_stemmed'

2. LEMMATIZATION (WordNet Lemmatizer)
------------------------------------------------------------
✓ Lemmatization completada. Columna: 'tokens_lemmatized'

3. COMPARACIÓN DE RESULTADOS Y VOCABULARIO
------------------------------------------------------------
Token Original       Stemming             Lemmatization       
------------------------------------------------------------
presentation         present              presentation        
material             materi               material            
viewed               view                 view                

Tamaño del vocabulario base: 30955
Tamaño después de Stemming: 26963 (12.90% reducción)
Tamaño después de Lemmatization: 29716 (4.00% reducción)

✓ Datos guardado

5. Representación Tradicional: Bag of Words (BoW)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de estilo
sns.set_theme(style="whitegrid", palette="viridis")

print("\n" + "=" * 60)
print("VECTORIZACIÓN: BAG OF WORDS (BoW) SIMPLIFICADA")
print("=" * 60)

# --- 1. CONFIGURACIÓN Y TRANSFORMACIÓN ---
print("\n1. CREANDO MATRIZ BAG OF WORDS")
print("-" * 60)

# El estudiante debe elegir la columna que mejor funcionó (lemmatized, stemmed, processed)
TEXT_COLUMN = 'text_lemmatized'

# Crear y configurar el vectorizador
bow_vectorizer = CountVectorizer(
    # Parámetros esenciales para reducir el ruido
    max_features=5000, 
    min_df=2, 
    max_df=0.8, 
    ngram_range=(1, 2)
)

# Ajustar (aprender vocabulario) y transformar (crear matriz)
X_bow = bow_vectorizer.fit_transform(df_processed[TEXT_COLUMN])
feature_names = bow_vectorizer.get_feature_names_out()

print(f"✓ Matriz BoW creada. Forma: {X_bow.shape}")
print(f"Tamaño del Vocabulario Final: {X_bow.shape[1]}")


# --- 2. ANÁLISIS DE FRECUENCIAS GLOBALES ---
print("\n2. ANÁLISIS DE FRECUENCIAS (TOP 15)")
print("-" * 60)

# Calcular frecuencias de todos los términos en un solo paso
word_freq = np.asarray(X_bow.sum(axis=0)).flatten()
freq_df = pd.DataFrame({
    'word': feature_names,
    'frequency': word_freq
}).sort_values('frequency', ascending=False).head(15)

print(freq_df)

# Visualización simple (usando Seaborn para un gráfico más estético)
plt.figure(figsize=(10, 6))
sns.barplot(x='frequency', y='word', data=freq_df)
plt.title(f'Top 15 Términos Más Frecuentes (BoW)', fontweight='bold')
plt.tight_layout()
plt.savefig('bow_top_terms_simple.png', dpi=300)
plt.show()


# --- 3. ANÁLISIS POR SENTIMIENTO (Simplificado) ---
print("\n3. TÉRMINOS CLAVE POR SENTIMIENTO (Top 5 por Clase)")
print("-" * 60)

# Recorrer cada sentimiento para obtener sus palabras clave
for sentiment in df_processed['Sentiment'].unique():
    # 1. Filtrar los documentos
    docs_sentiment = df_processed[df_processed['Sentiment'] == sentiment][TEXT_COLUMN]
    
    # 2. Vectorizar solo esos documentos (usando el vocabulario ya aprendido)
    X_sentiment = bow_vectorizer.transform(docs_sentiment)
    
    # 3. Calcular frecuencias
    freq_sentiment = np.asarray(X_sentiment.sum(axis=0)).flatten()
    
    # 4. Crear DataFrame y mostrar TOP 5
    freq_df_sentiment = pd.DataFrame({
        'word': feature_names,
        'frequency': freq_sentiment
    }).sort_values('frequency', ascending=False).head(5)
    
    print(f"\n{sentiment.upper()}:")
    print(freq_df_sentiment)


# --- 4. PREPARACIÓN FINAL PARA EL MODELO ---
print("\n4. PREPARACIÓN FINAL DE DATOS")
print("-" * 60)

# Guardar la matriz BoW con etiquetas (esencial para el entrenamiento)
# Creamos un DataFrame a partir de la matriz dispersa
X_dense = X_bow.toarray()
bow_final_df = pd.DataFrame(X_dense, columns=feature_names)

# Añadimos la columna de Sentimiento (Y) y el Idioma (si es necesario)
bow_final_df['Sentiment'] = df_processed['Sentiment'].values
bow_final_df['Idioma'] = df_processed['Idioma'].values

bow_final_df.to_csv('datos_vectorizados_final.csv', index=False)
print(f"✓ Matriz de características (X) y Sentimiento (Y) guardada en 'datos_vectorizados_final.csv'")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\aleja\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\aleja\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1043, in launch_instance
    app.start()
  File "C:\Users\aleja\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 728, in start
    self.io_lo

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

6. Representación Tradicional: TF-IDF

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
# ❌ Eliminadas: import matplotlib.pyplot as plt
# ❌ Eliminadas: import seaborn as sns

print("\n" + "=" * 60)
print("VECTORIZACIÓN: TF-IDF FUNCIONAL (SIN VISUALIZACIÓN)")
print("=" * 60)

# Elige la columna de texto limpia y normalizada
TEXT_COLUMN = 'text_lemmatized'

# --- 1. CREACIÓN Y TRANSFORMACIÓN TF-IDF ---
print("\n1. CREANDO MATRIZ TF-IDF")
print("-" * 60)

tfidf_vectorizer = TfidfVectorizer(
    max_features=5000,
    min_df=2,
    max_df=0.8,
    ngram_range=(1, 2),
    sublinear_tf=True
)

# Nota: El DataFrame df_processed debe estar definido
try:
    X_tfidf = tfidf_vectorizer.fit_transform(df_processed[TEXT_COLUMN])
except NameError:
    print("ERROR: El DataFrame 'df_processed' no está definido. Asegúrate de cargar los datos antes.")
    sys.exit() # Salir para evitar más errores

tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()

print(f"✓ Modelo TF-IDF creado. Forma: {X_tfidf.shape}")
print(f"Tamaño del Vocabulario Final: {X_tfidf.shape[1]}")


# --- 2. ANÁLISIS DE TÉRMINOS CLAVE (SIN GRÁFICOS) ---
print("\n2. TÉRMINOS CON MAYOR PESO TF-IDF PROMEDIO (TOP 15)")
print("-" * 60)

# Calcular el peso promedio de cada término
tfidf_means = np.asarray(X_tfidf.mean(axis=0)).flatten()
tfidf_df = pd.DataFrame({
    'term': tfidf_feature_names,
    'tfidf_mean': tfidf_means
}).sort_values('tfidf_mean', ascending=False).head(15)

print(tfidf_df)

# ❌ Eliminada la sección de plt.figure() y sns.barplot()


# --- 3. PREPARACIÓN FINAL PARA EL MODELO ---
print("\n3. PREPARACIÓN FINAL DE DATOS")
print("-" * 60)

# Guardar la matriz TF-IDF con etiquetas (X y Y)
tfidf_matrix_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_feature_names)
tfidf_matrix_df['Sentiment'] = df_processed['Sentiment'].values

tfidf_matrix_df.to_csv('datos_tfidf_final.csv', index=False)
print(f"✓ Matriz de características (X) y Sentimiento (Y) guardada en 'datos_tfidf_final.csv'")


VECTORIZACIÓN: TF-IDF FUNCIONAL (SIN VISUALIZACIÓN)

1. CREANDO MATRIZ TF-IDF
------------------------------------------------------------
✓ Modelo TF-IDF creado. Forma: (11684, 5000)
Tamaño del Vocabulario Final: 5000

2. TÉRMINOS CON MAYOR PESO TF-IDF PROMEDIO (TOP 15)
------------------------------------------------------------
         term  tfidf_mean
1625      eur    0.017926
1696    euros    0.016214
3400      per    0.011562
68       2008    0.009391
972   company    0.009384
77       2009    0.009337
1238      der    0.009115
61       2007    0.008641
1664     euro    0.008584
1296      die    0.008503
1621      eta    0.008433
3657   profit    0.008405
1251      des    0.008390
3971    sales    0.007952
3057      net    0.007597

3. PREPARACIÓN FINAL DE DATOS
------------------------------------------------------------
✓ Matriz de características (X) y Sentimiento (Y) guardada en 'datos_tfidf_final.csv'


7. Word Embeddings No Contextuales - Word2Vec

8. Word Embeddings No Contextuales - GloVe Pre-entrenado

9. Word Embeddings No Contextuales - FastText

10. Word Embeddings Contextuales - BERT

11. Embeddings Contextuales - Modelos Multilingües

12. Comparación de Todas las Representaciones

13. Estructura Final de Datos

14. Evaluación de Calidad de Embeddings